* Plot radio dynamic spectra from ORFEES and NenuFAR.
* Plot timeseries of STIX, ORFEES, and NenuFAR.

In [1]:
import warnings
warnings.filterwarnings('ignore')
import glob
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import astropy.units as u
from astropy.time import Time
from astropy.io import fits
from sunpy.net import Fido, attrs as a
from sunpy.timeseries import TimeSeries
from radiospectra.spectrogram2 import Spectrogram
from stixpy.net.client import STIXClient  # This registers the STIX client with Fido
from stixpy.product import Product
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from matplotlib.colors import LogNorm
from matplotlib.collections import QuadMesh
import matplotlib as mpl
mpl.rcParams.update({'font.size': 14})
mpl.rcParams['date.epoch'] = '1970-01-01T00:00:00' # use precise epoch
try: mdates.set_epoch('1970-01-01T00:00:00')
except: pass

# import time, os

data_dir = '/home/mnedal/data'

In [2]:
mydate = '2025-03-25'

year, month, day = mydate.split('-')

In [4]:
files = glob.glob(f'{data_dir}/int_orf{year}{month}{day}_*.fts')

orfees = fits.open(files[0])
orfees_i = np.hstack([orfees[2].data[f'STOKESI_B{i}'] for i in range(1, 6)]).T
orfees_data = orfees_i.T

In [ ]:



# Remove the background by taking the data from the quiet background and divide it by the data
for i in range(data.shape[0]):
	data[i] = data[i]/data[35000]

data = data.T
#pdb.set_trace()

orfees_time_str = orfees[0].header['DATE-OBS']
orfees_times = Time(orfees_time_str) + (orfees[2].data['TIME_B1']/1000)*u.s # times are not the same for all sub spectra!
orfees_freqs = np.hstack([orfees[1].data[f'FREQ_B{i}'] for i in range(1, 6)]) *u.MHz

orfees_meta = {
    'observatory': orfees[0].header['ORIGIN'],
    'instrument': orfees[0].header['INSTRUME'],
    'detector': orfees[0].header['INSTRUME'],
    'freqs': orfees_freqs.reshape(-1),
    'times': orfees_times,
    'wavelength': a.Wavelength(orfees_freqs[0,0], orfees_freqs[0,-1]),
    'start_time': orfees_times[0],
    'end_time': orfees_times[-1]
}

###### Plot the spec
orfees_spec_i = Spectrogram(data, orfees_meta)
#pdb.set_trace()
vmm = np.percentile(orfees_spec_i.data, [1,96])
fig = plt.figure(figsize=(12, 7))
ax = fig.add_subplot(111)
#orfees_spec_i.plot(axes = ax, norm=LogNorm(vmin=vmm[0], vmax=vmm[1]), cmap = 'Spectral_r')
orfees_spec_i.plot(axes = ax, vmin=vmm[0], vmax=vmm[1], cmap = 'Spectral_r')
ax.set_ylim(143,500)
plt.show()